In [1]:
import numpy as np
import time
import os

import matplotlib.pyplot as plt
import pandas as pd
import random

import json

import keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import models

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = 'false'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], False)

DEBUG = False

2022-01-30 16:17:31.876979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 16:17:31.904355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 16:17:31.904540: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']
file = '/home/joren/Coding/cryptodata/Normalized_labelled/FTTUSDT.bin'
modelfile = 'models/model_small_10'
frame_size = 320

In [3]:
# load data
field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data, dtype=np.float16)

In [4]:
# load model
model = models.load_model(modelfile)

2022-01-30 16:17:32.162127: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-30 16:17:32.162925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 16:17:32.163132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 16:17:32.163272: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
# train-test split
def df_split(df):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_test = []
    y_test = []
    for i in range(frame_size, X.shape[0]): 
        X_test.append(X[i-frame_size:i])
        y_test.append(Y[i])
    X_test, y_test = np.array(X_test, dtype=np.float16), np.array(y_test, dtype=np.float16)

    return X_test, y_test

In [6]:
## prepare data for prediction
data = read_bin_full_file(file)
df2 = pd.DataFrame(data, columns=column_names)
df2.fillna(0, inplace=True)
startloc = round(len(df2)*0.9)
df = df2.iloc[startloc:]
# df = df.iloc[round(len(df)*0.74):-2000]
del(df2)

In [7]:
X_test, y_test = df_split(df)

In [8]:
## predict
y_pred = np.array(model.predict(X_test), dtype=np.float16)

y_pred = np.argmax(y_pred, axis=-1)
y_pred = y_pred.flatten()

print(f"""
Class. report:
{classification_report(y_test, y_pred)}
""")
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

2022-01-30 16:17:55.346127: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300



Class. report:
              precision    recall  f1-score   support

         0.0       1.00      0.41      0.58     75940
         1.0       0.01      0.75      0.02       265
         2.0       0.01      0.82      0.01       264

    accuracy                           0.41     76469
   macro avg       0.34      0.66      0.21     76469
weighted avg       0.99      0.41      0.57     76469


[[30812 15640 29488]
 [   56   200     9]
 [   45     3   216]]
40.83746354731983


In [9]:
df = df.iloc[frame_size:]
df['predictions'] = np.array(y_pred)

In [10]:
print(df.head())

        timestamp      open     close      high       low    volume     adosc  \
691421        inf -0.000140  0.000245  0.000088 -0.000281 -0.641602 -0.064026   
691422        inf  0.000228 -0.001366 -0.001294 -0.000719 -0.749023 -0.051758   
691423        inf -0.001471 -0.000787 -0.001328 -0.001471  1.749023 -0.047974   
691424        inf -0.001311 -0.000472 -0.000506 -0.000175  2.183594 -0.044250   
691425        inf -0.001101 -0.002216 -0.001238 -0.001608 -0.282227 -0.036255   

             atr      macd  macd_signal  macd_hist       mfi  upper_band  \
691421 -0.001392 -0.018478    -0.034943   0.099365  0.649414    0.000116   
691422  0.012817  0.015198    -0.023956   0.423828  0.568359    0.000100   
691423  0.019211  0.070740    -0.004669   4.109375  0.548828   -0.000071   
691424  0.025391  0.048340     0.005692  -1.825195  0.616699   -0.000471   
691425  0.012566  0.079041     0.019241  -0.698242  0.602051   -0.000284   

        middle_band  lower_band       rsi  difference_lo

In [11]:
# calculate profit/loss
total_percentage_raw = 0
percentage = 0
position = 0

for index, row in df.iterrows():

    if position == 1:
        percentage += row['close']

    if row['predictions'] == 1:
        position = 1
    elif row['predictions'] == 2:
        position = 0
        # if percentage > 0:
        #     print(f'profit: {percentage}')
        # elif percentage < 0:
        #     print(f'loss: {percentage}')
        total_percentage_raw += percentage
        percentage = 0

print(total_percentage_raw)
    

2.629611849784851


In [12]:
# calculate profit/loss with skip
total_percentage_skip = 0
percentage = 0
position = 0
skip_buy = 0
skip_sell = 0
skipcounter = 2

for index, row in df.iterrows():

    if position == 1:
        percentage += row['close']

    if row['predictions'] == 1:
        if skip_buy == skipcounter:
            position = 1
            skip_buy = 0
        skip_buy += 1
    elif row['predictions'] == 2:
        if skip_sell == skipcounter:
            skip_sell = 0
            position = 0
            # if percentage > 0:
            #     print(f'profit: {percentage}')
            # elif percentage < 0:
            #     print(f'loss: {percentage}')
            total_percentage_skip += percentage
            percentage = 0
        skip_sell += 1

print(total_percentage_skip)

2.197593927383423


In [13]:
# calculate profit/loss only sell with profit
total_percentage_profit_only = 0
percentage = 0
position = 0

for index, row in df.iterrows():

    if position >= 1:
        percentage += row['close']

    if row['predictions'] == 1:
        position = 1
    elif row['predictions'] == 2:
        if percentage > 0.01:
            # print(f'profit: {percentage}')
            total_percentage_profit_only += percentage
            percentage = 0
            position = 0
        

print(total_percentage_profit_only)
    

1.0527297854423523


In [14]:
# save output results to json

with open('test_results/model_tests.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)

dictionary ={
    "file" : file,
    "model" : modelfile,
    # "skipcounter": skipcounter,
    "total_percentage_raw" : total_percentage_raw
    # "total_percentage_skip" : total_percentage_skip,
    # "total_percentage_profit_only": total_percentage_profit_only
}

# new_object = json.dumps(dictionary, indent = 4)
json_object['results'].append(dictionary)
json_object = json.dumps(json_object, indent = 4)
with open("test_results/model_tests.json", "w") as outfile:
    outfile.write(json_object)